In [ ]:
from confluent_kafka import Producer
import requests
import json
import time
import pandas as pd

url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    'vs_currency': 'usd',
    'order': 'market_cap_desc',
    'per_page': 100,
    'page': 1,
    'sparkline': False
}
# Configure the Kafka Producer
conf = {'bootstrap.servers': 'localhost:9092'}
producer = Producer(conf)

# Initialize an empty DataFrame
all_data = pd.DataFrame()

def delivery_report(err, msg):
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}]")

while True:
    response = requests.get(url, params=params)
    data = response.json()
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Append new data to the existing data
    all_data = pd.concat([all_data, df], ignore_index=True)
    
    # Save the data to a CSV file
    all_data.to_csv('kafka_crypto_data.csv', index=False)
    
    # Send the data to Kafka
    producer.produce('crypto-topic', json.dumps(data).encode('utf-8'), callback=delivery_report)
    producer.poll(1)  # Wait up to 1 second for events
    
    time.sleep(60)
